# Conexión a Bases de datos

**Autor:** Roberto Muñoz <br />
**E-mail:** <rmunoz@metricarts.com> <br />
**Github:** <https://github.com/rpmunoz> <br />


- Abrir Anaconda prompt
- Instalar el conector de MySQL para python
    `conda install -c anaconda mysql-connector-python`

In [1]:
import numpy as np
import pandas as pd
import mysql.connector as sql

In [2]:
# Database args
db_user = 'duoc'
db_passwd = 'duoc2017'
db_host = 'videoanalyticsvm6427.cloudapp.net'
db_name = 'chile'

In [4]:
cnx = sql.connect(user=db_user, password=db_passwd, host=db_host, database=db_name)

Usando el cursor de SQL. Ojo que antes de cerrar el cursor tenemos que realizar las operaciones de lectura que necesitemos.

In [5]:
query= "SHOW TABLES;"

cursor = cnx.cursor(buffered=True)
cursor.execute(query)

for el in cursor:
    print(el)

cursor.close()

('comunas',)
('provincias',)
('regiones',)


True

Usando el método read_sql de Pandas

In [6]:
query= "SHOW TABLES;"
tablas=pd.read_sql(query, con=cnx)
tablas

,Tables_in_chile
0,comunas
1,provincias
2,regiones


Hagamos un select en la tabla regiones

In [8]:
query= "SELECT * from regiones;"
regiones=pd.read_sql(query, con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,1,Arica y Parinacota,XV
1,2,Tarapacá,I
2,3,Antofagasta,II
3,4,Atacama,III
4,5,Coquimbo,IV
5,6,Valparaiso,V
6,7,Metropolitana de Santiago,RM
7,8,Libertador General Bernardo O'Higgins,VI
8,9,Maule,VII
9,10,Biobío,VIII


La siguiente consulta está parametrizada. Fíjese cómo se declaran los parámetros dentro de la cadena de la consulta, y cómo después se ingresan, en el comando read_sql de Pandas.

In [9]:
query= "SELECT * from regiones WHERE region_id BETWEEN %s and %s;"
regiones=pd.read_sql(query, params=(2,8), con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,2,Tarapacá,I
1,3,Antofagasta,II
2,4,Atacama,III
3,5,Coquimbo,IV
4,6,Valparaiso,V
5,7,Metropolitana de Santiago,RM
6,8,Libertador General Bernardo O'Higgins,VI


Como verá en la siguiente consulta, estamos utilizando cosas aprendidas en los tutoriales anteriores. Estamos armando dinámicamente la cadena que respresenta una consulta. Específicamente, utilizamos una lista con id's de regiones que nos interesan, lo convertimos a un objeto `array`de numpy (np.array), después aplicamos la función `map` para aplicar la función `str` al array, y finalmente, concatenamos los resultados con el caracter ",". **Todo en una línea**.

In [10]:
region_ids=[5,8,10]

query= "SELECT * from regiones WHERE region_id IN ("
query_region= ', '.join(map(str, np.array(region_ids)))
query= query + query_region + ");"

regiones=pd.read_sql(query, con=cnx)
regiones

,region_id,region_nombre,region_ordinal
0,5,Coquimbo,IV
1,8,Libertador General Bernardo O'Higgins,VI
2,10,Biobío,VIII


Cerramos la conexión al servidor de MySQL. **¡¡No olvide hacer esto!!**

In [11]:
cnx.close()